In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("atharvasoundankar/global-housing-market-analysis-2015-2024")
path_1 = kagglehub.dataset_download("gpandi007/usa-housing-dataset")
print("Path to dataset files:", path)

import pandas as pd
# Load the dataset
global_data = pd.read_csv(path + "/global_housing_market_extended.csv")
us_data = pd.read_csv(path_1 + "/housing_train.csv")



/Users/dolphins/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/dolphins/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/dolphins/.cache/kagglehub/datasets/atharvasoundankar/global-housing-market-analysis-2015-2024/versions/1


In [1]:
!pip install kagglehub


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [4]:
"""Joining Mortgage Rate, Inflation Rate, GDP Growth, and Population Growth scraped from Bankrate & Fed Reserve"""
econ_data = pd.DataFrame({
    "YrSold": [2006, 2007, 2008, 2009, 2010],
    "Mortgage Rate (%)": [6.41, 6.34, 6.03, 5.04, 4.69],
    "Inflation Rate (%)": [3.2, 2.8, 3.8, -0.4, 1.6],
    "GDP Growth (%)": [2.7, 1.8, -0.1, -2.5, 2.6],
    "Population Growth (%)": [0.89, 0.95, 0.99, 0.86, 0.88]
})

# Merge into your dataset
us_data = us_data.merge(econ_data, on="YrSold", how="left")

In [5]:
# Get rid of NA values
us_data.isnull().sum().sort_values(ascending=False).head(10)

# Drop columns with too many missing values or little importance
cols_to_drop = ['PoolQC', 'MiscFeature']
us_data.drop(columns=cols_to_drop, inplace=True)

# ---------------------------
# Fill 'None' for features where missing means "doesn't exist"
none_fill_cols = [
    'Alley', 'Fence', 'MasVnrType', 'FireplaceQu',
    'GarageQual', 'GarageFinish', 'GarageCond'
]
us_data[none_fill_cols] = us_data[none_fill_cols].fillna('None')

# ---------------------------
# Impute LotFrontage using median by Neighborhood
us_data['LotFrontage'] = us_data.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.median())
)


In [6]:
# Total Bathrooms: full + half (each half bath = 0.5)
us_data['TotalBathrooms'] = (
    us_data['FullBath'] +
    0.5 * us_data['HalfBath'] +
    us_data['BsmtFullBath'] +
    0.5 * us_data['BsmtHalfBath']
)

# Total square footage: 1st + 2nd floor + basement
us_data['TotalSF'] = us_data['1stFlrSF'] + us_data['2ndFlrSF'] + us_data['TotalBsmtSF']

# Age of house when sold and time since remodel
us_data['HouseAge'] = us_data['YrSold'] - us_data['YearBuilt']
us_data['RemodelAge'] = us_data['YrSold'] - us_data['YearRemodAdd']

# Binary flags
us_data['HasBasement'] = (us_data['TotalBsmtSF'] > 0).astype(int)
us_data['HasGarage'] = (us_data['GarageArea'] > 0).astype(int)


In [7]:
# Build Random Forest Model

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Drop identifier
df = us_data.drop(columns=["Id"])

# Define target
y = df["SalePrice"]

# Drop target from features
X = df.drop(columns=["SalePrice"])

# Label encode all object (categorical) columns
for col in X.select_dtypes(include="object").columns:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

X = X.fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

# Manually calculate RMSE
mse = ((y_test - y_pred) ** 2).mean()
rmse = mse ** 0.5

print(f"Manual RMSE: {rmse:.2f}")

Manual RMSE: 29388.97


In [ ]:
# Feature Importance Plots
import matplotlib.pyplot as plt

importances = pd.Series(rf.feature_importances_, index=X.columns)
importances.sort_values(ascending=False).head(20).plot(kind='barh', figsize=(10, 6))
plt.title("Top 20 Feature Importances")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
!pip install xgboost


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Assume X and y are already prepped like before
X = df.drop(columns=["SalePrice"])
y = df["SalePrice"]

# Label encode categoricals (same as before)
from sklearn.preprocessing import LabelEncoder
for col in X.select_dtypes(include='object').columns:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# Handle missing values
X = X.fillna(0)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build model
model_xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model_xgb.fit(X_train, y_train)

# Predict and evaluate manually
y_pred = model_xgb.predict(X_test)
rmse = ((y_test - y_pred) ** 2).mean() ** 0.5
print(f"XGBoost RMSE: {rmse:.2f}")